In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
# import matplotlib.cbook as cbook
# import matplotlib.dates as mdates


# Data

In [2]:
import math
import pandas as pd
import os.path
from datetime import timedelta, datetime
from dateutil import parser
from binance.client import Client as Client_binance


class BinanceClient(object):

    def __init__(self,
                 config):
        self.binance_api_key = config['binance_api_key']
        self.binance_api_secret = config['binance_api_secret']

        self.client = Client_binance(api_key=self.binance_api_key, api_secret=self.binance_api_secret)

    ### FUNCTIONS
    def minutes_of_new_data(self, symbol, initial_date,
                            kline_size, data, source):
        if len(data) > 0:
            old = parser.parse(data["timestamp"].iloc[-1])
        elif source == "binance":
            old = datetime.strptime(initial_date, '%d %b %Y')
        if source == "binance":
            new = pd.to_datetime(self.client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
        return old, new
    
    def get_all_binance(self, symbol, initial_date, freq, save=False):
        binsizes = {"1m": 1, "5m": 5, "10m": 10, "15m": 15, "1h": 60, "6h": 360, "12h": 720, "1d": 1440}
        filename = '/home/agustin/Git-Repos/algo-trading-crypto/il-hedger/il_hedger/files/%s-%s-data.csv' % (symbol, freq)
        if os.path.isfile(filename):
            data_df = pd.read_csv(filename)
        else:
            data_df = pd.DataFrame()
        oldest_point, newest_point = self.minutes_of_new_data(symbol, initial_date,
                                                              freq, data_df, source="binance")
        delta_min = (newest_point - oldest_point).total_seconds() / 60
        available_data = math.ceil(delta_min / binsizes[freq])
        if oldest_point == datetime.strptime(initial_date, '%d %b %Y'):
            print('Downloading all available %s data for %s. Be patient..!' % (freq, symbol))
        else:
            print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.'
                  % (delta_min, symbol, available_data, freq))
        klines = self.client.get_historical_klines(symbol, freq,
                                                   oldest_point.strftime("%d %b %Y %H:%M:%S"),
                                                   newest_point.strftime("%d %b %Y %H:%M:%S"))
        data = pd.DataFrame(klines,
                            columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
                                     'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
        data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
        if len(data_df) > 0:
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
        else:
            data_df = data
        data_df.set_index('timestamp', inplace=True)
        if save:
            data_df.to_csv(filename)
        print('All caught up..!')
        return data_df

import json

with open('/home/agustin/Git-Repos/HedgingScripts/files/StgyApp_config.json') as json_file:
    config = json.load(json_file)
_binance_client_ = BinanceClient(config['binance_client'])


# Track historical data
symbol_eth = 'ETHBUSD'
symbol_btc = 'BTCBUSD'
freq = '1d'
initial_date = "1 Jan 2021"
_binance_client_ = BinanceClient(config['binance_client'])
eth_historical = _binance_client_.get_all_binance(symbol=symbol_eth, freq=freq,
                              initial_date=initial_date, save=False)
btc_historical = _binance_client_.get_all_binance(symbol=symbol_btc, freq=freq,
                              initial_date=initial_date, save=False)
eth_prices = eth_historical['close']
for i in range(len(eth_prices)):
    eth_prices[i] = float(eth_prices[i])
btc_prices = btc_historical['close']
for i in range(len(btc_prices)):
    btc_prices[i] = float(btc_prices[i])

All caught up..!
All caught up..!


<ipython-input-2-6a9e21f37cc2>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_prices[i] = float(eth_prices[i])
<ipython-input-2-6a9e21f37cc2>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_prices[i] = float(btc_prices[i])


In [3]:
# data
# data = btc_historical
data_eth_2021 = pd.DataFrame(eth_prices).loc["2021-01-01":"2022-01-01"]
data_btc_2021 = pd.DataFrame(btc_prices).loc["2021-01-01":"2022-01-01"]

data_eth_2022 = pd.DataFrame(eth_prices).loc["2022-01-01":"2023-01-01"]
data_btc_2022 = pd.DataFrame(btc_prices).loc["2022-01-01":"2023-01-01"]

# Note that last dataset in each set of periods has a length of less than one expect for that period, 
# so is better to take all the periods except the last one. ie [:-1]

data_7days_eth_2021 = [data_eth_2021[i:i+7] for i in range(len(data_eth_2021)) if i % 7 == 0][:-1]
data_7days_btc_2021 = [data_btc_2021[i:i+7] for i in range(len(data_btc_2021)) if i % 7 == 0][:-1]

data_7days_eth_2022 = [data_eth_2022[i:i+7] for i in range(len(data_eth_2022)) if i % 7 == 0][:-1]
data_7days_btc_2022 = [data_btc_2022[i:i+7] for i in range(len(data_btc_2022)) if i % 7 == 0][:-1]

data_15days_eth_2021 = [data_eth_2021[i:i+15] for i in range(len(data_eth_2021)) if i % 15 == 0][:-1]
data_15days_btc_2021 = [data_btc_2021[i:i+15] for i in range(len(data_btc_2021)) if i % 15 == 0][:-1]

data_15days_eth_2022 = [data_eth_2022[i:i+15] for i in range(len(data_eth_2022)) if i % 15 == 0][:-1]
data_15days_btc_2022 = [data_btc_2022[i:i+15] for i in range(len(data_btc_2022)) if i % 15 == 0][:-1]

data_30days_eth_2021 = [data_eth_2021[i:i+30] for i in range(len(data_eth_2021)) if i % 30 == 0][:-1]
data_30days_btc_2021 = [data_btc_2021[i:i+30] for i in range(len(data_btc_2021)) if i % 30 == 0][:-1]

data_30days_eth_2022 = [data_eth_2022[i:i+30] for i in range(len(data_eth_2022)) if i % 30 == 0][:-1]
data_30days_btc_2022 = [data_btc_2022[i:i+30] for i in range(len(data_btc_2022)) if i % 30 == 0][:-1]

In [4]:
# Import pandas
import pandas as pd

# Load the xlsx file
parameters_eth_7_eur = pd.read_excel('Parameters/Orbit-Cruize_eth_eur.xlsx', sheet_name='7')
parameters_eth_15_eur = pd.read_excel('Parameters/Orbit-Cruize_eth_eur.xlsx', sheet_name='15')
parameters_eth_30_eur = pd.read_excel('Parameters/Orbit-Cruize_eth_eur.xlsx', sheet_name='30')

parameters_eth_7_amer = pd.read_excel('Parameters/Orbit-Cruize_eth_amer.xlsx', sheet_name='7')
parameters_eth_15_amer = pd.read_excel('Parameters/Orbit-Cruize_eth_amer.xlsx', sheet_name='15')
parameters_eth_30_amer = pd.read_excel('Parameters/Orbit-Cruize_eth_amer.xlsx', sheet_name='30')

parameters_btc_7_eur = pd.read_excel('Parameters/Orbit-Cruize_btc_eur.xlsx', sheet_name='7')
parameters_btc_15_eur = pd.read_excel('Parameters/Orbit-Cruize_btc_eur.xlsx', sheet_name='15')
parameters_btc_30_eur = pd.read_excel('Parameters/Orbit-Cruize_btc_eur.xlsx', sheet_name='30')

parameters_btc_7_amer = pd.read_excel('Parameters/Orbit-Cruize_btc_amer.xlsx', sheet_name='7')
parameters_btc_15_amer = pd.read_excel('Parameters/Orbit-Cruize_btc_amer.xlsx', sheet_name='15')
parameters_btc_30_amer = pd.read_excel('Parameters/Orbit-Cruize_btc_amer.xlsx', sheet_name='30')


In [5]:
eth_7_ascent_parameters_eur = pd.DataFrame(parameters_eth_7_eur.iloc[3:8]) 
eth_7_ascent_parameters_eur.columns = ['eth', 'Lower Barrier', 'Upper Barrier', 'Base Coupon','Participation Rate', 'Bonus Coupon','Max APR']

eth_7_twin_win_parameters_eur = parameters_eth_7_eur.iloc[8:13]
eth_7_twin_win_parameters_eur.columns = ['eth', 'Lower Barrier', 'Upper Barrier', 'Base Coupon','Participation Rate', 'Bonus Coupon','Max APR']

eth_7_highland_parameters_eur = parameters_eth_7_eur.iloc[13:17]
eth_7_highland_parameters_eur.columns = ['eth', 'Lower Barrier', 'Upper Barrier', 'Base Coupon','Participation Rate', 'Bonus Coupon','Max APR']

eth_7_corridor_parameters = parameters_eth_7_eur.iloc[17:21]
eth_7_corridor_parameters.columns = ['eth', 'Lower Barrier', 'Upper Barrier', 'Base Coupon','Participation Rate', 'Bonus Coupon','Max APR']

In [34]:
eth_30_ascent_parameters_eur.loc[8] = [0,0.7,1.05,0,0,0.017,0]

NameError: name 'eth_30_ascent_parameters_eur' is not defined

In [13]:
eth_7_ascent_parameters_eur

,eth,Lower Barrier,Upper Barrier,Base Coupon,Participation Rate,Bonus Coupon,Max APR
3,Ascent,0.98,1.02,0.005,7,0.28,0.285
4,NaN,0.96,1.04,0.005,1.8,0.144,0.149
5,NaN,0.94,1.06,0.005,0.85,0.102,0.107
6,NaN,0.92,1.08,0.005,0.55,0.088,0.093
7,NaN,0.9,1.1,0.005,0.25,0.05,0.055
8,0.0,0.7,1.05,0.0,0.0,0.017,0.0


In [4]:
def convert_to_pcg(data_set):
    for col in list(data_set.columns):
        for i in list(data_set.index):
            if (i == 'APY') | (i == 'APR') | (i == 'MAX_APY') | (i == 'MAX_APR'):
                data_set[col][i] = str(round((data_set[col][i]-1)*100,3)) + '%'
            else:
                data_set[col][i] = str(round(data_set[col][i]*100,3)) + '%'
    return data_set

## KIKO

In [28]:
def KIKO(week, KI, KO, P_last):
    strike = week['close'][0]
    knocked_out = False
    activated = False
    for P in week['close']:
        if P > KO:
            knocked_out = True
        elif KI > P:
            activated = True
    if P_last > KO:
        knocked_out = True
    if (not knocked_out) and activated:
        return max(strike/P_last-1, 0) # put
    else:
        return 0

In [40]:
def KIKO_backtest(data_periods, parameters_period):
    index = ['period_'+str(i) for i in range(len(data_periods))]
    other_parameters = ['Avg', 'Max', 'APY', 'APR', 'MAX_APY', 'MAX_APR']
    index += other_parameters
    periods_in_one_year = len(data_periods)#-1 # We remove last period in data_periods bc it has less than len(period) prices
    # index_parameters = list(eth_30_ascent_parameters.index)
    columns = [str([down_barrier, up_barrier]) for down_barrier, up_barrier in 
               zip(parameters_period['Lower Barrier'],#.loc[index_parameters[-1:]],
                   parameters_period['Upper Barrier'])]+[str([down_barrier, up_barrier])+'_APR' for down_barrier, up_barrier in 
               zip(parameters_period['Lower Barrier'],
                   parameters_period['Upper Barrier'])]+[str([down_barrier, up_barrier])+'_APY' for down_barrier, up_barrier in 
               zip(parameters_period['Lower Barrier'],
                   parameters_period['Upper Barrier'])]
    KIKO_period_results = pd.DataFrame(index=index, 
                          columns = columns)
    for down_barrier, up_barrier, premium in zip(parameters_period['Lower Barrier'],
                                                  parameters_period['Upper Barrier'],
                                                  parameters_period['Bonus Coupon']):
        for i in range(len(data_periods)):
            period = data_periods[i]
            S_0=period['close'][0]
            KI=S_0*down_barrier
            KO=S_0*up_barrier
            if i<len(data_periods)-1:
                P_last = data_periods[i+1]['close'][0]
            else:
                P_last = period['close'][-1]
            period_performance_as_pcg = KIKO(period, KI, KO, P_last)
            KIKO_period_results[str([down_barrier, up_barrier])].iloc[i] = (premium - period_performance_as_pcg) #/periods_in_one_year
            # APR, APY
            KIKO_period_results[str([down_barrier, up_barrier]) + '_APR'].iloc[i] = KIKO_period_results[str([down_barrier, up_barrier])].iloc[i] *periods_in_one_year
            KIKO_period_results[str([down_barrier, up_barrier]) + '_APY'].iloc[i] = (KIKO_period_results[str([down_barrier, up_barrier])].iloc[i] +1)**periods_in_one_year-1
#             print(period_performance_as_pcg)
    KIKO_period_results.iloc[len(index)-len(other_parameters)] = np.mean(KIKO_period_results)
    KIKO_period_results.iloc[len(index)-len(other_parameters)+1] = np.max(KIKO_period_results)
    KIKO_period_results.iloc[len(index)-len(other_parameters)+2] = (1+KIKO_period_results).cumprod().iloc[len(index)-len(other_parameters)-1]
    KIKO_period_results.iloc[len(index)-len(other_parameters)+3] = 1+KIKO_period_results.cumsum().iloc[len(index)-len(other_parameters)-1]
    KIKO_period_results.iloc[len(index)-len(other_parameters)+4] = (KIKO_period_results.iloc[len(index)-len(other_parameters)+1]+1)** periods_in_one_year
    KIKO_period_results.iloc[len(index)-len(other_parameters)+5] = 1+KIKO_period_results.iloc[len(index)-len(other_parameters)+1]* periods_in_one_year
    return KIKO_period_results

In [36]:
eth_7_ascent_parameters_eur.loc[8:9]

,eth,Lower Barrier,Upper Barrier,Base Coupon,Participation Rate,Bonus Coupon,Max APR
8,0.0,0.7,1.05,0.0,0.0,0.017,0.0


In [39]:
KIKO_backtest(data_30days_eth_2021, eth_7_ascent_parameters_eur.loc[8:9])

0
0
0
0
0
0
0
0
0
0
0
0


/home/agustin/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/agustin/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


,"[0.7, 1.05]","[0.7, 1.05]_APR","[0.7, 1.05]_APY"
period_0,0.017,0.204,0.224197
period_1,0.017,0.204,0.224197
period_2,0.017,0.204,0.224197
period_3,0.017,0.204,0.224197
period_4,0.017,0.204,0.224197
period_5,0.017,0.204,0.224197
period_6,0.017,0.204,0.224197
period_7,0.017,0.204,0.224197
period_8,0.017,0.204,0.224197
period_9,0.017,0.204,0.224197


## Vanilla Products (w/o barriers)

In [ ]:
# from https://quant.stackexchange.com/questions/57780/issue-in-pricing-binary-options-using-heaviside-function-and-quantlib-python
import QuantLib as ql

today = ql.Date().todaysDate()
initialValue = S
riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(today, r_f, ql.Actual365Fixed()))
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0, ql.Actual365Fixed()))
volatility = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), sigma, ql.Actual365Fixed()))

process = ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(initialValue)), riskFreeTS, dividendTS, volatility)
steps = 1
rng = "pseudorandom" # could use "lowdiscrepancy"
numPaths = 500000
option_type = ql.Option.Put
strike_price = S*upper_pcg

# option_type = ql.Option.Call
# strike_price = S*down_pcg

maturity_date = today+ ql.Period(7, ql.Days)#ql.Date(30, 1, 2023)
exercise = ql.EuropeanExercise(maturity_date)

payoff_cash = ql.CashOrNothingPayoff(option_type, strike_price, 1)
binary_option_cash = ql.VanillaOption(payoff_cash, exercise)

payoff_asset = ql.AssetOrNothingPayoff(option_type, strike_price)
binary_option_asset = ql.VanillaOption(payoff_asset, exercise)

payoff_vanilla=ql.PlainVanillaPayoff(option_type, strike_price)
binary_option_vanilla = ql.VanillaOption(payoff_vanilla, exercise)

engine = ql.MCEuropeanEngine(process, rng, steps, requiredSamples=numPaths)

In [43]:
import opstrat as op
P_entry = data_7days_eth_2021[0]
K_call = P_entry * 1.15
K_put = P_entry * 0.9

# premium_call_long = 167.6
# premium_call_short = -104.06
# premium_put_long = 96.7

sigma = data_7days_eth_2021.pct_change().std()

bsm_long_call=op.black_scholes(K=K_call, St=P_entry, r=4, t=7, 
                     v=sigma, type='c')
bsm_short_call=op.black_scholes(K=K_call, St=P_entry, r=4, t=15, 
                     v=sigma*2, type='c')
bsm_long_put=op.black_scholes(K=K_call, St=P_entry, r=4, t=30, 
                     v=sigma*4, type='c')

AttributeError: 'list' object has no attribute 'pct_change'

# Number of breaches

In [44]:
ranges = [0.02, 0.05, 0.08, 0.1]
pct_changes_weekly_comaprison = pd.DataFrame(index = [str(round(rang*100,3))+'%' for rang in ranges], columns = ['ETH', 'BTC'])
number_of_breaches = pd.DataFrame(index = [str(round(rang*100,3))+'%' for rang in ranges], columns = ['ETH', 'BTC'])
for rang in ranges:
    pct_changes_weekly_comaprison['ETH'].loc[str(round(rang*100,3))+'%'] = np.mean(np.abs(data_eth['close'][::7].pct_change())>rang)
    number_of_breaches['ETH'].loc[str(round(rang*100,3))+'%'] = sum(np.abs(data_eth['close'][::7].pct_change())>rang)/len(data_eth['close'][::7])
    pct_changes_weekly_comaprison['BTC'].loc[str(round(rang*100,3))+'%'] = np.mean(np.abs(data_btc['close'][::7].pct_change())>rang)
    number_of_breaches['BTC'].loc[str(round(rang*100,3))+'%'] = sum(np.abs(data_btc['close'][::7].pct_change())>rang)/len(data_btc['close'][::7])
##################################################
pct_change_by_week_eth = pd.DataFrame([0]*len(ranges),
                                  index = [str(round(rang*100,3))+'%' for rang in ranges])
for j in range(len(ranges)):
    rang = ranges[j]
    week_change = []
#     for i in range(len(data_weeks_btc)):
    for i in range(len(data_weeks_eth)):
#         week = data_weeks_btc[i]
        week = data_weeks_eth[i]
        week_change.append(week['close'][-1]/week['close'][0]-1>rang)
    pct_change_by_week_eth.iloc[j] = np.mean(week_change)

NameError: name 'data_eth' is not defined